In [9]:
import string
import numpy as np

In [10]:
def one_hot(n_classes: int, idx: int) -> np.ndarray:
    embedding = np.zeros(n_classes)
    embedding[idx] = 1.0
    return embedding


def word2seq(word: str, alphabet: list) -> np.ndarray:
    return np.array([one_hot(len(alphabet), alphabet.index(c)) for c in word])


def seq2word(seq: np.ndarray, alphabet: list) -> str:
    return ''.join([alphabet[np.argmax(x)] for x in seq])

In [34]:
alphabet = string.ascii_letters + " " + string.punctuation + string.digits
n_symbols = len(alphabet)
print(alphabet)

abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~0123456789


In [12]:
def softmax(x: np.ndarray) -> np.ndarray:
    y = np.exp(x)
    return y / np.sum(y)

In [ ]:
class EncoderRNN:
    def __init__(self, n_in: int, n_h: int) -> None:
        self.U: np.ndarray = np.random.uniform(-1, 1, (n_h, n_in))
        self.V: np.ndarray = np.random.uniform(-1, 1, (n_h, n_h))
        self.d: np.ndarray = np.zeros(n_h)
    
    def forward(self, sequence: np.ndarray) -> np.ndarray:
        context = np.zeros(len(self.V))
        for x in sequence:
            context = np.tanh(self.U @ x + self.V @ context + self.d)
        return context

In [14]:
class DecoderRNN:
    def __init__(self, n_in: int, n_h: int, n_out: int) -> None:
        self.U: np.ndarray = np.random.uniform(-1, 1, (n_h, n_in))
        self.V: np.ndarray = np.random.uniform(-1, 1, (n_h, n_h))
        self.d: np.ndarray = np.zeros(n_h)

        self.W: np.ndarray = np.random.uniform(-1, 1, (n_out, n_h))
        self.b: np.ndarray = np.zeros(n_out)
    
    def forward(self, context: np.ndarray, t: int) -> np.ndarray:
        outputs = np.zeros((t, self.b.size))
        out = np.zeros(self.b.size)

        for i in range(t):
            context = np.tanh(self.U @ out + self.V @ context + self.d)
            out = softmax(self.W @ context + self.b)
            outputs[i] = out.copy()
        
        return outputs

In [33]:
n_h = 32

encoder = EncoderRNN(n_symbols, n_h)
decoder = DecoderRNN(n_symbols, n_h, n_symbols)

in_text = 'the quick brown fox jumps over the lazy dog'
seq = word2seq(in_text, alphabet)

context = encoder.forward(seq)
output = decoder.forward(context, 128)

out_text = seq2word(output, alphabet)
print(out_text)

C'^4l*lf@|*x8=flX8kI=fyX*2Lk8kXyid\8x*-ac)R2}AK/qI|8xfka!Z8\*/f'x8xUkfyXLkfka;|8kbl]@\8xfWa'x}2}-fkf!^8kfyaa"EEEEXgXbWvW.&2}2*-0
